---
* **Universidad:** UOC (Universitat Oberta de Catalunya)
* **Titulación:** Master Ciencia de Datos/TFM
* **Título:** Segmentación de imágenes utilizando Redes Generativas Antagónicas
* **Actividad**: Segmentación partes cuerpo personas con armas cortas utilizando modelos pre entrenados
* **Autor:** Miguel Ponce
* **Correo:** miguelalejoponce@uoc.edu
* **Fecha:** "Enero, 04 de 2023"
* **Salida**: ipynb
---
## Contexto: Detección de personas

* **Contexto:**
Se desea segmentar las personas y armas cortas con los modelos pre entrenados.

* **Problema:** 
Es necesrio realizar la segmentación de las personas armadas con armas cortas, para tener un conjunto de imagenes que luego serán procesadas para ser utilizadas como conjunto de datos para la red GAN.

* **Solucion :** 
Se utilizara los modelos pre entrenados de las etapas anteriores para generar un nuevo conjunto de datos. Para esto se hara uso de los conjuntos de datos tratado con super pixeles ESRGAN.
* **Entorno:** con GPU.
  

## Origen del la fuente de datos
* **ESRGAN:**

https://raw.githubusercontent.com/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-esrgan-personas-armadas.part1.rar

https://raw.githubusercontent.com/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-esrgan-personas-armadas.part2.rar


## Modelos pre entrenados
* **DETECCIÓN ARMA:**
https://drive.google.com/file/d/1xAbxae-WI2uiTPUOZvakSNhvbkNBc2sT/view

* **SEGMENTACIÓN ARMA:** https://drive.google.com/file/d/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob/view

* **SEGMENTACIÓN PERSONA:** https://github.com/de-code/python-tf-bodypix

# Descarga conjunto de datos

In [1]:
!wget https://raw.githubusercontent.com/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-esrgan-personas-armadas.part1.rar
!wget https://raw.githubusercontent.com/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-esrgan-personas-armadas.part2.rar

--2023-01-07 04:22:44--  https://raw.githubusercontent.com/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-esrgan-personas-armadas.part1.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94371840 (90M) [application/octet-stream]
Saving to: ‘super-resolution-esrgan-personas-armadas.part1.rar’

super-resolution-es 100%[===================>]  90.00M   454MB/s    in 0.2s    

2023-01-07 04:22:44 (454 MB/s) - ‘super-resolution-esrgan-personas-armadas.part1.rar’ saved [94371840/94371840]

--2023-01-07 04:22:44--  https://raw.githubusercontent.com/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-esrgan-personas-armadas.part2.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199

In [2]:
!unrar x super-resolution-esrgan-personas-armadas.part1.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from super-resolution-esrgan-personas-armadas.part1.rar

Creating    super-resolution-esrgan-personas-armadas                  OK
Creating    super-resolution-esrgan-personas-armadas/super-resolution-esrgan-tf2  OK
Creating    super-resolution-esrgan-personas-armadas/super-resolution-esrgan-tf2/person  OK
Extracting  super-resolution-esrgan-personas-armadas/super-resolution-esrgan-tf2/person/10948387_452160258269315_769342041_n_jpg.rf.37f4964a48434944e9c3d7970b10fe2f.jpg       0%  OK 
Extracting  super-resolution-esrgan-personas-armadas/super-resolution-esrgan-tf2/person/10948387_452160258269315_769342041_n_jpg.rf.6f1f24908ceb8df134009fbc99230f3c.jpg       0%  OK 
Extracting  super-resolution-esrgan-personas-armadas/super-resolution-esrgan-tf2/person/10948387_452160258269315_769342041_n_jpg.rf.98cb658213ec9cd37068b34e4aba517c.jpg       0%  OK 
Extracting  super-resolution-esrgan-p

In [3]:
!cp -r ./super-resolution-esrgan-personas-armadas/super-resolution-esrgan-tf2 ./
!rm -rf ./super-resolution-esrgan-personas-armadas

# Deteccion Armas Cortas

## Instalar requisitos

_(Recuerde elegir GPU en tiempo de ejecución si aún no lo ha seleccionado. Tiempo de ejecución --> Cambiar tipo de tiempo de ejecución --> Acelerador de hardware --> GPU)_

In [4]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-0rt8bu_j
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-0rt8bu_j
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9f71f50b13c76e95540a5944bfa00c49db55eab0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 14.9 MB/s eta 0:00:0

## Importar Librerias



In [5]:
import os
import glob
import time
import torch
from pathlib import Path
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_hub as hub
import logging
import torch
from collections import OrderedDict
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.utils.events import EventStorage
from detectron2.modeling import build_model
import detectron2.utils.comm as comm
from detectron2.engine import default_argument_parser, default_setup, default_writers, launch
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    inference_on_dataset,
    print_csv_format,
)

## Mostrar caracteristicas del entorno

In [6]:
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
torch:  1.13 ; cuda:  cu116
detectron2: 0.6


## Constantes

In [7]:
# Declaring Constants
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"
TEST_MODE = False
MAX_IMAGES = 100
mainPath = "./"

## Clases


In [8]:
class DetectImage:
  def __init__(self,sourcePath,outputPath,config,modelName,threshold) -> None:
    self.cfg = config
    print(os.path.join(self.cfg.OUTPUT_DIR, modelName))
    self.cfg.MODEL.WEIGHTS = os.path.join(self.cfg.OUTPUT_DIR, modelName)
    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    self.predictor = DefaultPredictor(self.cfg)
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def crop_object(self,imagePath):
    """Crops an object in an image

    Inputs:
      image: PIL image
      box: one box from Detectron2 pred_boxes
    """  
    image = Image.open(imagePath)
    im = cv2.imread(imagePath)
    outputs = self.predictor(im)
    boxes = outputs["instances"].pred_boxes
    # Select 1 box:    
    if list(boxes):
      #print(imagePath)
      #print(boxes,"boxes")
      box = list(boxes)[0].detach().cpu().numpy()         
      x_top_left = box[0]
      y_top_left = box[1]
      x_bottom_right = box[2]
      y_bottom_right = box[3]
      x_center = (x_top_left + x_bottom_right) / 2
      y_center = (y_top_left + y_bottom_right) / 2
      crop_img = image.crop((int(x_top_left), int(y_top_left), int(x_bottom_right), int(y_bottom_right)))
      #plt.figure()
      #plt.axis('off')
      #plt.imshow(crop_img) 
      #plt.show() 
      return True,crop_img,box
    else:
      return False,None,None

  def saveBox(self,box,boxPath):
    with open(boxPath, "w") as f:
      for s in box:
        f.write(str(s) +"\n")
  def readBox(self,boxPath):
    values = []
    with open(boxPath, "r") as f:
      for line in f:
        values.append(float(line.strip()))
    return values
   
  
  def processImages(self):       
    path = self.sourcePath+"/*.jpg"
    print(path)
    images = glob.glob(path)
    index = 0
    start = time.time()
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1             
      hasBox,crop_img,box = self.crop_object(imagePath)
      if hasBox:
        imageName = os.path.basename(imagePath)
        print(imageName)
        name = Path(imagePath).stem        
        boxPath = os.path.join(self.outputPath,"boxes",name+".txt")
        cropImagePath = os.path.join(self.outputPath,"crops",imageName)
        crop_img.save(cropImagePath)
        self.saveBox(box,boxPath)
        print("Time Detect Image : %f" % (time.time() - start))

## Configuracion modelos pre entrenados

In [9]:
from detectron2.engine import DefaultTrainer
instance = "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(instance))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(instance)  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## Descaga modelo detección arma pre-entrenado

In [10]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1" -O ./output/model_final.pth && rm -rf /tmp/cookies.txt

--2023-01-07 04:26:25--  https://docs.google.com/uc?export=download&confirm=t&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1
Resolving docs.google.com (docs.google.com)... 74.125.200.139, 74.125.200.100, 74.125.200.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.200.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i108ghnmp52iej2vmnfm9svcuiamf6gn/1673065575000/07156903365772177219/*/1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1?e=download&uuid=f1516d72-6477-4e98-b414-2bbda6f510db [following]
--2023-01-07 04:26:26--  https://doc-0g-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i108ghnmp52iej2vmnfm9svcuiamf6gn/1673065575000/07156903365772177219/*/1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1?e=download&uuid=f1516d72-6477-4e98-b414-2bbda6f510db
Resolving doc-0g-bk-docs.googleusercontent.com (doc-0g-bk-docs.googleusercontent.com)... 142.251.12.132, 

## Creacion proyecto deteccion arma 
ESRGAN

In [11]:
!rm -rf guns-detection-esrgan
!mkdir guns-detection-esrgan
!mkdir "guns-detection-esrgan/crops"
!mkdir "guns-detection-esrgan/boxes"
!mkdir "guns-detection-esrgan/binmask"
!mkdir "guns-detection-esrgan/binredmask"

## Creacion Pipeline red pre entrenada detección arma 
ESRGAN

In [12]:
sourcePathESRGAN = os.path.join(mainPath,"super-resolution-esrgan-tf2/person")
outputPathESRGAN = os.path.join(mainPath,"guns-detection-esrgan")
detectImage = DetectImage(sourcePathESRGAN,outputPathESRGAN,cfg,"model_final.pth",0.85)
detectImage.processImages()

./output/model_final.pth
[01/07 04:26:43 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...
./super-resolution-esrgan-tf2/person/*.jpg


/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


gmd_1_frame75_jpg.rf.2d0952816c9eaa3fd914a0c53ee2edb4.jpg
Time Detect Image : 6.995872
gmd_5_frame6_jpg.rf.e707dcfb14f659fde24df071a0fd448c.jpg
Time Detect Image : 7.262896
gmd_4_frame70_jpg.rf.0c94762ba7f74097359f9ada07ca6e27.jpg
Time Detect Image : 7.516039
gmd_1_frame208_jpg.rf.6d6d76892fb2122d09c8616b7a8a3735.jpg
Time Detect Image : 7.761617
10948387_452160258269315_769342041_n_jpg.rf.37f4964a48434944e9c3d7970b10fe2f.jpg
Time Detect Image : 8.111231
gmd_5_frame20_jpg.rf.6e41657eac837ee7a321835813a0f67e.jpg
Time Detect Image : 8.577122
423450861-1607013005_jpeg_jpg.rf.dfdb948014e3c866b610a90f8c016200.jpg
Time Detect Image : 10.262855
mgd_varying_frame108_jpg.rf.eec33f616f5d8bb5e56c19051341354e.jpg
Time Detect Image : 10.657716
youtube_frame150_jpg.rf.0b652a685d49c38aebb9f9e047680d44.jpg
Time Detect Image : 10.985879
mgd_varying_frame188_jpg.rf.2d6394acae486d6208cafe968e26a323.jpg
Time Detect Image : 11.354194
mgd_varying_frame56_jpg.rf.e50ff0309d37711bbfbad3b95197e727.jpg
Time Detec

# Segmentacion Armas

## Clases

In [13]:
class SegmentImage:
  def __init__(self,sourcePath,outputPath,config,modelName,threshold) -> None:
    self.cfg = config
    self.cfg.MODEL.WEIGHTS = os.path.join(self.cfg.OUTPUT_DIR, modelName)
    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    self.predictor = DefaultPredictor(self.cfg)
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def crop_shape_object(self,imagePath):
    """Crops an object in an image

    Inputs:
      image: PIL image
      box: one box from Detectron2 pred_boxes
    """  
   
    img= cv2.imread(imagePath)
    outputs= self.predictor(img)
    if outputs['instances']: 
      mask= outputs['instances'].get('pred_masks')    
      mask= mask.to('cpu')    
      num, h, w= mask.shape
      bin_mask= np.zeros((h, w))
      index  = 0
      for m in mask:        
          if index > 0:
            break;
          bin_mask = m.numpy().astype(np.uint8)
          #indices = indices.astype(np.uint8)  #convert to an unsigned byte
          bin_mask*=255
          #print(bin_mask.any(0))
          #print(index)
          index = index +1
          #bin_mask.append(m)
      return True,bin_mask
    return False,None
  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePath+"/*.jpg"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1                
      hasMask,bin_mask = self.crop_shape_object(imagePath)
      if hasMask:
        imageName = os.path.basename(imagePath)
        print(imageName)
        imageNamePng = Path(imageName).stem+'.png'
        im = Image.fromarray(bin_mask)
        imagePngPath = os.path.join(self.outputPath,imageNamePng)                        
        im.save(imagePngPath, "PNG")
        print("Time Segment Image : %f" % (time.time() - start))

## Configuracion modelo segementación pre entrenado

In [14]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("asl_poly_train",)#Train dataset registered in a previous cell
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00055
cfg.SOLVER.MAX_ITER = 1200 
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  
cfg.TEST.EVAL_PERIOD = 0 # Increase this number if you want to monitor validation performance during training

PATIENCE = 500 #Early stopping will occur after N iterations of no imporovement in total_loss

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## Descaga modelo segemntación arma pre-entrenado

In [15]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob" -O ./output/model_final_segmentaion.zip && rm -rf /tmp/cookies.txt

--2023-01-07 04:32:29--  https://docs.google.com/uc?export=download&confirm=t&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob
Resolving docs.google.com (docs.google.com)... 142.250.4.113, 142.250.4.138, 142.250.4.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.4.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t71lq7c37m7pr4t1tiqvp0kmimiuqcr9/1673065875000/07156903365772177219/*/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob?e=download&uuid=e6ac3c62-390e-43e3-b7a9-7da15d5c67cc [following]
--2023-01-07 04:32:29--  https://doc-08-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t71lq7c37m7pr4t1tiqvp0kmimiuqcr9/1673065875000/07156903365772177219/*/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob?e=download&uuid=e6ac3c62-390e-43e3-b7a9-7da15d5c67cc
Resolving doc-08-bk-docs.googleusercontent.com (doc-08-bk-docs.googleusercontent.com)... 142.251.12.132, 2404

In [16]:
!unzip -o ./output/model_final_segmentaion.zip -d ./output/

Archive:  ./output/model_final_segmentaion.zip
  inflating: ./output/model_final_segmentaion.pth  


## Creacion Pipeline red pre entrenada segmentación arma ESRGAN

In [17]:
sourcePathCropsESRGAN = os.path.join(mainPath,"guns-detection-esrgan/crops")
outputPathBinMaskESRGAN = os.path.join(mainPath,"guns-detection-esrgan/binmask")
segmentImage = SegmentImage(sourcePathCropsESRGAN,outputPathBinMaskESRGAN,cfg,"model_final_segmentaion.pth",0.60)
segmentImage.processImages()

[01/07 04:32:35 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final_segmentaion.pth ...
gmd_1_frame75_jpg.rf.2d0952816c9eaa3fd914a0c53ee2edb4.jpg
Time Segment Image : 0.215498
gmd_5_frame6_jpg.rf.e707dcfb14f659fde24df071a0fd448c.jpg
Time Segment Image : 0.351043
gmd_4_frame70_jpg.rf.0c94762ba7f74097359f9ada07ca6e27.jpg
Time Segment Image : 0.450424
gmd_1_frame208_jpg.rf.6d6d76892fb2122d09c8616b7a8a3735.jpg
Time Segment Image : 0.552819
10948387_452160258269315_769342041_n_jpg.rf.37f4964a48434944e9c3d7970b10fe2f.jpg
Time Segment Image : 0.696062
gmd_5_frame20_jpg.rf.6e41657eac837ee7a321835813a0f67e.jpg
Time Segment Image : 0.825056
423450861-1607013005_jpeg_jpg.rf.dfdb948014e3c866b610a90f8c016200.jpg
Time Segment Image : 0.931079
mgd_varying_frame108_jpg.rf.eec33f616f5d8bb5e56c19051341354e.jpg
Time Segment Image : 1.035107
youtube_frame150_jpg.rf.0b652a685d49c38aebb9f9e047680d44.jpg
Time Segment Image : 1.178246
mgd_varying_frame188_jpg.rf.2d63

# Segemtancion partes del cuerpo

## Instalar requisitos

_(Recuerde elegir GPU en tiempo de ejecución si aún no lo ha seleccionado. Tiempo de ejecución --> Cambiar tipo de tiempo de ejecución --> Acelerador de hardware --> GPU)_

In [18]:
!pip uninstall -y detectron2 
!pip uninstall -y tensorflow
!pip uninstall -y tensorflow_decision_forests
!rm -rf build/ **/*.so

Found existing installation: detectron2 0.6
Uninstalling detectron2-0.6:
  Successfully uninstalled detectron2-0.6
Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Successfully uninstalled tensorflow-2.9.2


In [19]:
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 tf_bodypix opencv-python tfjs_graph_converter matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [20]:
!python -m pip install --upgrade imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Clases

In [21]:
class BodyPix:
  def __init__(self,sourcePath,outputPath) -> None:
    #self.bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_FLOAT_50_STRIDE_16))
    self.bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_RESNET50_FLOAT_STRIDE_16))
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def maskImage(self,imagePath):
    imageName = os.path.basename(imagePath)
    
    # load model (once)

    image = tf.keras.utils.load_img(imagePath)
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    # get prediction result
    #image = tf.keras.preprocessing.image.load_img(fullPath)    
    result = self.bodypix_model.predict_single(image_array)

    # simple mask
    mask = result.get_mask(threshold=0.75)
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-mask",imageName),
        mask)

    # colored mask (separate colour for each body part)
    colored_mask = result.get_colored_part_mask(mask)
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-colored-mask",imageName),
        colored_mask)



    poses = result.get_poses()
    image_with_poses = draw_poses(
        image_array.copy(),  # create a copy to ensure we are not modifing the source image
        poses,
        keypoints_color=(255, 100, 100),
        skeleton_color=(100, 100, 255)
    )
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-poses",imageName),
        image_with_poses
    )
    

  def processImages(self):       
    start = time.time() 
    path = self.sourcePath+"/*.jpg"
    print(path)
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1
      imageName = os.path.basename(imagePath)
      print(imageName)      
      self.maskImage(imagePath) 
      print("Time Taken Body Pix : %f" % (time.time() - start))

## Liberias BodyPix

In [22]:
import tensorflow as tf
from tf_bodypix.api import download_model, load_model, BodyPixModelPaths
from tf_bodypix.draw import draw_poses  # utility function using OpenCV

## Creacion proyecto body pix ESRGAN

In [23]:
!rm -rf body-pix-esrgan
!mkdir body-pix-esrgan
!mkdir "body-pix-esrgan/person"
!mkdir "body-pix-esrgan/person/output-mask"
!mkdir "body-pix-esrgan/person/output-colored-mask"
!mkdir "body-pix-esrgan/person/output-poses"

## Creacion Pipeline red detección partes cuerpo ESRGAN

In [24]:
sourcePathMask = os.path.join(mainPath,"super-resolution-esrgan-tf2/person")
outputPathBody = os.path.join(mainPath,"body-pix-esrgan/person")
bodyPix = BodyPix(sourcePathMask,outputPathBody)
bodyPix.processImages()

./super-resolution-esrgan-tf2/person/*.jpg
gmd_1_frame75_jpg.rf.2d0952816c9eaa3fd914a0c53ee2edb4.jpg
Time Taken Body Pix : 4.644726
gmd_5_frame6_jpg.rf.e707dcfb14f659fde24df071a0fd448c.jpg
Time Taken Body Pix : 5.930880
gmd_4_frame70_jpg.rf.0c94762ba7f74097359f9ada07ca6e27.jpg
Time Taken Body Pix : 7.175711
gmd_1_frame208_jpg.rf.6d6d76892fb2122d09c8616b7a8a3735.jpg
Time Taken Body Pix : 8.393324
10948387_452160258269315_769342041_n_jpg.rf.37f4964a48434944e9c3d7970b10fe2f.jpg
Time Taken Body Pix : 9.764450
gmd_1_frame22_jpg.rf.64fa6735bb57bf8025fe3aa8d35bb37d.jpg
Time Taken Body Pix : 11.022091
gmd_5_frame20_jpg.rf.6e41657eac837ee7a321835813a0f67e.jpg
Time Taken Body Pix : 12.242128
desi-pistol-image-holding-hands_55_jpeg_jpg.rf.f9c23bbc6f28ef5953b5beb0a5b0658c.jpg
Time Taken Body Pix : 16.381023
mgd_custom_frame2770_jpg.rf.2d008f3c8cd6a4f3843c3b9558587ecb.jpg
Time Taken Body Pix : 17.342219
mgd_custom_frame1744_jpg.rf.c17a31c8930b885066fdc84801aa2362.jpg
Time Taken Body Pix : 18.360685

# Reemplazo mascara 

## Librerias

In [25]:
import shutil

## Clases

In [26]:
class BitMaskGenerator:
  def __init__(self,sourcePath,outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def createBitMask(self,imagePath):
    img = Image.open(imagePath)
    img = img.convert("RGBA")
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255,0,0, 0))
        else:
            newData.append(item)

    img.putdata(newData)
    return img.convert('RGB')
    

    return False,None
  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePath+"/*.png"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1                      
      rgb_im = self.createBitMask(imagePath)    
      imageName = os.path.basename(imagePath)      
      print(imageName)
      imageNamePng = Path(imageName).stem+'.png'      
      imagePngPath = os.path.join(self.outputPath,imageNamePng) 
      rgb_im.save(imagePngPath,"PNG")     
      print("Time BitMaskRed Image : %f" % (time.time() - start))

In [27]:
class ImageBitMaskUtil:
  def __init__(self,sourcePath,pathBitMask, pathBitRedMask, pathBox,outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath
    self.pathBitMask = pathBitMask
    self.pathBox = pathBox
    self.pathBitRedMask = pathBitRedMask
    
  def readBox(self,boxPath):
    values = []
    with open(boxPath, "r") as f:
      for line in f:
        values.append(float(line.strip()))
    return values
  def pasteBitMask(self,pathImage,format):
    imageName = os.path.basename(pathImage)
    name  = Path(imageName).stem
    pathImageMask = os.path.join(self.pathBitMask,name+".png")
    pathBitRedMask = os.path.join(self.pathBitRedMask,name+".png")
    if os.path.exists(pathImageMask):
      print(imageName)      
      boxImagePath  = os.path.join(self.pathBox,name+'.txt')
      box = self.readBox(boxImagePath)      
      print("box",box)
      with Image.open(pathImage) as imageBody:
          imageBody.load()
      with Image.open(pathBitRedMask) as imgGunRedMask:
          imgGunRedMask.load()
      with Image.open(pathImageMask) as imgGunMask:
          imgGunMask.load()
    
      imageBody.paste(
          imgGunRedMask.resize((imgGunRedMask.width, imgGunRedMask.height)),
          (int(box[0]) ,int(box[1])),
          imgGunMask.resize((imgGunMask.width, imgGunMask.height)),
      )
      if format == "png":
        imageNamePng = Path(imageName).stem+'.png' 
        pathImageResult = os.path.join(self.outputPath,imageNamePng)
        imageBody.save(pathImageResult,"PNG")
      else:
        pathImageResult = os.path.join(self.outputPath,imageName)
        imageBody.save(pathImageResult)
  
  def processImages(self,format):   
    start = time.time()    
    path = self.sourcePath+"/*.jpg"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1               
      imageName = os.path.basename(imagePath)      
      self.pasteBitMask(imagePath,format)             
      print("Time PasteBitMaskRed into Image : %f" % (time.time() - start))

In [28]:
class ImageSegmentedUtil:
  def __init__(self,sourcePathSegmented,sourcePath, outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath
    self.sourcePathSegmented = sourcePathSegmented    

  def processImages(self):   
    start = time.time()    
    path = self.sourcePathSegmented+"/*.png"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1        
      imageName = os.path.basename(imagePath)
      imageNameJpg = Path(imageName).stem+'.jpg' 
      print(imageName)
      originalImagePath = os.path.join(self.sourcePath,imageNameJpg) 
      outputImagePath  = os.path.join(self.outputPath,imageNameJpg) 
      shutil.copyfile(originalImagePath, outputImagePath)           
      print("Time PasteBitMaskRed into Image : %f" % (time.time() - start))

## Creacion proyecto merge armas segmentadas y personas

In [29]:
!rm -rf segmented-body-parts-gun-esrgan
!mkdir segmented-body-parts-gun-esrgan
!mkdir "segmented-body-parts-gun-esrgan/original"
!mkdir "segmented-body-parts-gun-esrgan/segmentado"
!mkdir "segmented-body-parts-gun-esrgan/mascara"

## Creacion Pipeline procesamiento merge partes cuerpo segmentadas y armas

In [30]:
sourcePatBinMask = os.path.join(mainPath,"guns-detection-esrgan/binmask")
outputPathBinRedMask = os.path.join(mainPath,"guns-detection-esrgan/binredmask")
bitMaskGenerator = BitMaskGenerator(sourcePatBinMask,outputPathBinRedMask)
bitMaskGenerator.processImages()

gmd_1_frame147_jpg.rf.2aa7de3b9aabc3692ec28ec9a74371ff.png
Time BitMaskRed Image : 0.038893
mgd_varying_frame110_jpg.rf.33fd7ecacac796d031272fb555796ab6.png
Time BitMaskRed Image : 0.141128
mgd_varying_frame222_jpg.rf.0f5f8b607257f02398928c7943acf4242.png
Time BitMaskRed Image : 0.173570
mgd_varying_frame49_jpg.rf.ef40ab199516652820a8612b5d6e7f5b.png
Time BitMaskRed Image : 0.329052
mgd_varying_frame187_jpg.rf.a9df5c41afdd6079f9656b0b91c4cb70.png
Time BitMaskRed Image : 0.406926
425458913-1607521221_jpeg_jpg.rf.7d44e1a91b7b2c4d6863ebdc44c95fac2.png
Time BitMaskRed Image : 0.542679
mgd_custom_frame2660_jpg.rf.e0e6b8134c6682f615b5ab9284f3aa55.png
Time BitMaskRed Image : 0.549786
Untitled-1-33_jpg.rf.4b139f45591b0844387b6bee571a55a1.png
Time BitMaskRed Image : 0.581607
gmd_3_frame42_jpg.rf.afbc5c68395c84c02756f48dedb8b689.png
Time BitMaskRed Image : 0.588041
406057747-1602603154_jpeg_jpg.rf.f3fd29f7ce43653ad1a4628229f8543b.png
Time BitMaskRed Image : 0.599701
mgd_varying_frame45_jpg.rf.71

In [31]:
sourcePathBody = "body-pix-esrgan/person/output-colored-mask"
pathBitMask = "guns-detection-esrgan/binmask"
pathBitRedMask = "guns-detection-esrgan/binredmask"
boxPath = "guns-detection-esrgan/boxes"
outputPathSegmented = "segmented-body-parts-gun-esrgan/segmentado"
imageBitMaskUtil = ImageBitMaskUtil(sourcePathBody,pathBitMask, pathBitRedMask, boxPath,outputPathSegmented)
imageBitMaskUtil.processImages("jpg")

gmd_1_frame75_jpg.rf.2d0952816c9eaa3fd914a0c53ee2edb4.jpg
box [709.2361, 319.88712, 818.7761, 493.43478]
Time PasteBitMaskRed into Image : 0.050263
gmd_5_frame6_jpg.rf.e707dcfb14f659fde24df071a0fd448c.jpg
box [585.16736, 168.86809, 701.7276, 351.59344]
Time PasteBitMaskRed into Image : 0.090540
gmd_4_frame70_jpg.rf.0c94762ba7f74097359f9ada07ca6e27.jpg
box [2.0607028, 898.58154, 125.41238, 1031.7635]
Time PasteBitMaskRed into Image : 0.124186
gmd_1_frame208_jpg.rf.6d6d76892fb2122d09c8616b7a8a3735.jpg
box [20.462738, 599.3651, 150.51118, 715.7519]
Time PasteBitMaskRed into Image : 0.154499
10948387_452160258269315_769342041_n_jpg.rf.37f4964a48434944e9c3d7970b10fe2f.jpg
box [622.1802, 186.93626, 839.718, 540.911]
Time PasteBitMaskRed into Image : 0.197915
Time PasteBitMaskRed into Image : 0.198419
gmd_5_frame20_jpg.rf.6e41657eac837ee7a321835813a0f67e.jpg
box [478.17062, 222.23438, 580.16034, 412.96774]
Time PasteBitMaskRed into Image : 0.230992
Time PasteBitMaskRed into Image : 0.231815
T

In [32]:
sourcePathBody = "body-pix-esrgan/person/output-mask"
pathBitMask = "guns-detection-esrgan/binmask"
pathBitRedMask = "guns-detection-esrgan/binmask"
boxPath = "guns-detection-esrgan/boxes"
outputPathSegmented = "segmented-body-parts-gun-esrgan/mascara"
imageBitMaskUtil = ImageBitMaskUtil(sourcePathBody,pathBitMask, pathBitRedMask, boxPath,outputPathSegmented)
imageBitMaskUtil.processImages("png")

gmd_1_frame75_jpg.rf.2d0952816c9eaa3fd914a0c53ee2edb4.jpg
box [709.2361, 319.88712, 818.7761, 493.43478]
Time PasteBitMaskRed into Image : 0.035292
gmd_5_frame6_jpg.rf.e707dcfb14f659fde24df071a0fd448c.jpg
box [585.16736, 168.86809, 701.7276, 351.59344]
Time PasteBitMaskRed into Image : 0.058314
gmd_4_frame70_jpg.rf.0c94762ba7f74097359f9ada07ca6e27.jpg
box [2.0607028, 898.58154, 125.41238, 1031.7635]
Time PasteBitMaskRed into Image : 0.079021
gmd_1_frame208_jpg.rf.6d6d76892fb2122d09c8616b7a8a3735.jpg
box [20.462738, 599.3651, 150.51118, 715.7519]
Time PasteBitMaskRed into Image : 0.100501
10948387_452160258269315_769342041_n_jpg.rf.37f4964a48434944e9c3d7970b10fe2f.jpg
box [622.1802, 186.93626, 839.718, 540.911]
Time PasteBitMaskRed into Image : 0.127154
Time PasteBitMaskRed into Image : 0.127264
gmd_5_frame20_jpg.rf.6e41657eac837ee7a321835813a0f67e.jpg
box [478.17062, 222.23438, 580.16034, 412.96774]
Time PasteBitMaskRed into Image : 0.146246
Time PasteBitMaskRed into Image : 0.146353
T

In [33]:
outputOriginalPath = "segmented-body-parts-gun-esrgan/original"
imageSegmentedUtil = ImageSegmentedUtil(outputPathSegmented,sourcePathMask, outputOriginalPath)
imageSegmentedUtil.processImages()

gmd_1_frame147_jpg.rf.2aa7de3b9aabc3692ec28ec9a74371ff.png
Time PasteBitMaskRed into Image : 0.003449
mgd_varying_frame110_jpg.rf.33fd7ecacac796d031272fb555796ab6.png
Time PasteBitMaskRed into Image : 0.003754
mgd_varying_frame222_jpg.rf.0f5f8b607257f02398928c7943acf4242.png
Time PasteBitMaskRed into Image : 0.008656
mgd_varying_frame49_jpg.rf.ef40ab199516652820a8612b5d6e7f5b.png
Time PasteBitMaskRed into Image : 0.009253
mgd_varying_frame187_jpg.rf.a9df5c41afdd6079f9656b0b91c4cb70.png
Time PasteBitMaskRed into Image : 0.010090
425458913-1607521221_jpeg_jpg.rf.7d44e1a91b7b2c4d6863ebdc44c95fac2.png
Time PasteBitMaskRed into Image : 0.011174
mgd_custom_frame2660_jpg.rf.e0e6b8134c6682f615b5ab9284f3aa55.png
Time PasteBitMaskRed into Image : 0.011471
Untitled-1-33_jpg.rf.4b139f45591b0844387b6bee571a55a1.png
Time PasteBitMaskRed into Image : 0.016479
gmd_3_frame42_jpg.rf.afbc5c68395c84c02756f48dedb8b689.png
Time PasteBitMaskRed into Image : 0.017629
406057747-1602603154_jpeg_jpg.rf.f3fd29f7c

In [34]:
!cp -r guns-detection-esrgan segmented-body-parts-gun-esrgan

## Compresion Archivos

In [35]:
!rm -rf segmented-body-parts-gun-esrgan.tar.gz
!tar -czvf segmented-body-parts-gun-esrgan.tar.gz "./segmented-body-parts-gun-esrgan"

Se han truncado las últimas 5000 líneas del flujo de salida.
./segmented-body-parts-gun-esrgan/original/mgd_custom_frame1618_jpg.rf.065f8afc89964e8ab630525ce23cbca9.jpg
./segmented-body-parts-gun-esrgan/original/mgd_custom_frame1038_jpg.rf.8e74e34b3ca9d31d3ec7dd5ef6639f7d.jpg
./segmented-body-parts-gun-esrgan/original/gmd_1_frame193_jpg.rf.51ab4b9faba2f027712de5b718a74165.jpg
./segmented-body-parts-gun-esrgan/original/494052653-1625580345_jpg.rf.1c6dc9981fb851fb3aa3a8ed21103eb2.jpg
./segmented-body-parts-gun-esrgan/original/gmd_5_frame37_jpg.rf.adc8139267e82ce512bd943261602f67.jpg
./segmented-body-parts-gun-esrgan/original/mgd_varying_frame193_jpg.rf.feb4d176a448e80904fc43ef62aef107.jpg
./segmented-body-parts-gun-esrgan/original/gmd_1_frame245_jpg.rf.92b459180a009dbde0173dcc3337284d.jpg
./segmented-body-parts-gun-esrgan/original/439976447-1611403992_jpeg_jpg.rf.a4f0a9202ab5d0ddb17e230a8fad05012.jpg
./segmented-body-parts-gun-esrgan/original/435302427-1610179658_jpeg_jpg.rf.983368380777

## Cargar en drive resultados

In [36]:
import os
from google.colab import drive
ROOT_FOLDER='/content/'
ROOT_DRIVE_FOLDER=ROOT_FOLDER+'drive/'
drive.mount(ROOT_DRIVE_FOLDER)

Mounted at /content/drive/


In [37]:
!cp --suffix=_v_$(date +"%y-%m-%d--%T")  ./segmented-body-parts-gun-esrgan.tar.gz /content/drive/MyDrive/tmf-person-data/data_set_processed/segmented-body-parts-gun-esrgan